# Assignment 3: All Pairs Similarity

## Setup & Dataset Retrieval

In [1]:
import os.path
import pickle
import random
from tqdm import tqdm

import src.utilities.io as io
import src.utilities.preprocess as pp
import src.apds.docsim_sequential as ds_seq
import src.apds.docsim_mapreduce_spark as ds_mr

/Users/a/miniforge3/envs/sbruch-assignment-Lwmd/lib/python3.10/site-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
[nltk_data] Downloading package punkt to /Users/a/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/a/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/a/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
documents, data_path = io.get_data() #trec-covid as default

2023-06-01 02:18:59 - Loading Corpus...


  0%|          | 0/171332 [00:00<?, ?it/s]

2023-06-01 02:19:02 - Loaded 171332 TEST Documents.
2023-06-01 02:19:02 - Doc Example: {'text': 'OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneumonia, 14 (35%) with upper respiratory tract 

filter out the documents with less than 50 words.

In [3]:
documents = {doc_id: doc for doc_id, doc in documents.items() if len(doc['text'].split()) >= 50 and doc['text'].strip() != ''}

In [4]:
for doc_id, doc in list(documents.items())[:10]:
    num_words = len(doc["text"].split())
    print(f"Document {doc_id}: {num_words} words")

Document ug7v899j: 262 words
Document 02tnwd4m: 142 words
Document ejv2xln0: 219 words
Document 2b73a28n: 68 words
Document 9785vg6d: 110 words
Document zjufx4fo: 174 words
Document 5yhe786e: 113 words
Document 8zchiykl: 83 words
Document 8qnrcgnk: 188 words
Document jg13scgo: 189 words


### Sort the documents by their length (Longest to Shortest)

In [5]:
sort_docs = sorted(documents.items(), key=lambda x: len(x[1]['text']), reverse=True)
sorted_docs: pp.Documents = {}
for doc_id, doc in sort_docs:
    sorted_docs[doc_id] = doc

In [6]:
len(sorted_docs)

124791

In [7]:
for doc_id, doc in list(sorted_docs.items())[:50]:
    num_words = len(doc["text"].split())
    print(f"Document {doc_id}: {num_words} words")

Document ij3ncdb6: 18000 words
Document c4pt07zk: 15448 words
Document 1vimqhdp: 12745 words
Document pd1g119c: 4642 words
Document gvh0wdxn: 3507 words
Document jkrj0lbm: 3783 words
Document jkvr9lq2: 3679 words
Document y00pz4fd: 1705 words
Document lqdgvsaq: 1627 words
Document vslevssr: 1504 words
Document o1vxgzpj: 1437 words
Document bt59txxp: 1474 words
Document yt9rj42s: 1460 words
Document qapwb5e4: 1529 words
Document rff3ikak: 1490 words
Document 0prx11md: 1310 words
Document 84hh0mg9: 1312 words
Document uumzl37q: 1053 words
Document vr7vm64u: 1097 words
Document v45cb2xq: 1136 words
Document yn7pu9i8: 1204 words
Document t473tbm9: 1271 words
Document nq0qe8qf: 1151 words
Document 16b8drw2: 1163 words
Document 4cbjbo7t: 1236 words
Document tvs1snq8: 1139 words
Document 2wb007gf: 1102 words
Document st1epx6u: 1114 words
Document syl7wq15: 1114 words
Document dt3li3bk: 1054 words
Document ec798c6s: 1057 words
Document xvnxkcxq: 1032 words
Document rxvxf34z: 1039 words
Documen

In [8]:
for doc_id, doc in list(sorted_docs.items())[-50:]:
    num_words = len(doc["text"].split())
    print(f"Document {doc_id}: {num_words} words")

Document tx2ps3j7: 53 words
Document fy177rd8: 50 words
Document z3tmy8hw: 51 words
Document nfoqst26: 50 words
Document cfdfjwdz: 50 words
Document uq4eobf0: 50 words
Document 1o8qlqln: 50 words
Document emzz23tn: 53 words
Document zrepv7ao: 50 words
Document ai9jh0xu: 50 words
Document ydqec1q1: 50 words
Document 1mbs9ch6: 56 words
Document eapkdq7v: 51 words
Document cuclkx06: 51 words
Document 3n6loyxt: 50 words
Document 10r5p0nu: 51 words
Document wlnbilr5: 51 words
Document g8e5sqh7: 51 words
Document 80ebtfwr: 50 words
Document vpouyyev: 50 words
Document za18znu4: 51 words
Document dntrv3op: 53 words
Document iutcduix: 50 words
Document yjxthqzi: 53 words
Document rzee3nm2: 52 words
Document u4e5tifa: 50 words
Document 8axi3ub9: 50 words
Document ljak6cmb: 53 words
Document e3vsaj8i: 50 words
Document e3b4eqy6: 50 words
Document inu9ykjc: 53 words
Document pxymyqoy: 50 words
Document kuwxzao3: 52 words
Document 1h3uzj49: 51 words
Document 5j4oz8ds: 51 words
Document 9k7owm1v: 5

This dictionary is used to map the index of a document to the corresponding text_id in the original dataset.
This index-to-ID mappings is useful for keeping track of the original text_id of the documents.

In [9]:
doc_idx_to_id = {i: doc_id for i, doc_id in enumerate(sorted_docs.keys())}

## Due to the size, we will use different dataset samples
(1000 longest docs, 1000 shortest docs, 1000 random docs)

### Define the paths to save and read the samples and their mappings

In [10]:
sample_size = 500
samples_dir = os.path.join(data_path, "samples_" + str(sample_size))
if not os.path.exists(samples_dir):
    os.mkdir(samples_dir)

random_docs_idx_path = os.path.join(samples_dir, "random_docs_idx.pkl")

In [11]:
if not os.path.exists(random_docs_idx_path):
    random_docs_idx = random.sample(range(len(sorted_docs)), k=sample_size)

    with open(random_docs_idx_path, "wb") as f:
        pickle.dump(random_docs_idx, f)
else:
    with open(random_docs_idx_path, "rb") as f:
        random_docs_idx = pickle.load(f)

longest_docs = pp.Documents = {}
for doc_id, doc in list(sorted_docs.items())[:sample_size]:
    longest_docs[doc_id] = doc
longest_doc_idx_to_id = {i:doc_idx_to_id[idx] for i, idx in enumerate(range(sample_size))}

shortest_docs = pp.Documents = {}
for doc_id, doc in list(sorted_docs.items())[-sample_size:]:
    shortest_docs[doc_id] = doc
shortest_doc_idx_to_id = {i:doc_idx_to_id[idx] for i, idx in enumerate(range(len(sorted_docs)-sample_size, len(sorted_docs)))}

random_docs = pp.Documents = {}
s_docs = list(sorted_docs.items())
for index in random_docs_idx:
    doc_id, doc = s_docs[index]
    random_docs[doc_id] = doc
random_doc_idx_to_id = {i:doc_idx_to_id[idx] for i, idx in enumerate(random_docs_idx)}

Mapping is useful if we want to retrieve the original documents corresponding to the randomly chosen documents in random_docs

In [12]:
samples = [("longest_docs", longest_docs, longest_doc_idx_to_id),
           ("shortest_docs", shortest_docs, shortest_doc_idx_to_id),
           ("random_docs", random_docs, random_doc_idx_to_id)]

## Tokenization
Document cleaning and tokenization

In [13]:
tokens_dir = os.path.join(samples_dir, "tokens")
if not os.path.exists(tokens_dir):
    os.mkdir(tokens_dir)

docs_tokens_path = os.path.join(tokens_dir, "doc_tokens.pkl")

In [14]:
if not os.path.exists(docs_tokens_path):
    tokenized_samples = []
    for name, docs, idx_to_id in samples:
        tokenized_docs: pp.TokenizedDocuments = pp.get_tokenized_documents(docs)
        tokenized_samples.append((name, tokenized_docs, idx_to_id))

    with open(docs_tokens_path, "wb") as f:
        pickle.dump(tokenized_samples, f)
else:
    with open(docs_tokens_path, "rb") as f:
        tokenized_samples = pickle.load(f)

Tokenizing documents: 100%|██████████| 500/500 [00:01<00:00, 498.41it/s]


## VECTORIZE

In [15]:
tfidf_dir = os.path.join(samples_dir, "tfidf")
if not os.path.exists(tfidf_dir):
    os.mkdir(tfidf_dir)

tfidf_results_path = os.path.join(tfidf_dir, "tfidf_docs.pkl")

In [16]:
if not os.path.exists(tfidf_results_path):
    tfidf_docs = []
    for name, docs, idx_to_id in tqdm(tokenized_samples):
        vectorized_docs, time_taken = pp.vectorize(docs)
        tfidf_docs.append((name, vectorized_docs, time_taken, idx_to_id))

    with open(tfidf_results_path, "wb") as f:
        pickle.dump(tfidf_docs, f)
else:
    with open(tfidf_results_path, "rb") as f:
        tfidf_docs = pickle.load(f)

100%|██████████| 3/3 [00:00<00:00,  6.43it/s]


In [17]:
for name, _, time_taken, _ in tfidf_docs:
    print(f"Time taken to vectorize {name}: {time_taken:.6f} seconds")

Time taken to vectorize longest_docs: 0.355644 seconds
Time taken to vectorize shortest_docs: 0.026516 seconds
Time taken to vectorize random_docs: 0.067268 seconds


## Setup

In [18]:
thresholds = [0.3, 0.5, 0.7, 0.9] # Chosen thresholds

## Sequential APDS

In [19]:
seq_results_dir = os.path.join(samples_dir, "seq_result")
if not os.path.exists(seq_results_dir):
    os.mkdir(seq_results_dir)

seq_results_path = os.path.join(seq_results_dir, "seq_results.pkl")

In [20]:
#sample: pairs indices, pairs docs id, additional information
all_seq_results = ds_seq.exec_seq_apds(tfidf_docs=tfidf_docs,
                                       thresholds=thresholds,
                                       seq_results_path=seq_results_path,
                                       samples_dir=samples_dir,
                                       heuristic=False)

100%|██████████| 3/3 [00:01<00:00,  2.82it/s]


### longest 1000

In [21]:
ds_seq.print_results(all_seq_results, "longest_docs")

--Run info--
sample_name: longest_docs
threshold: 0.3
pairs_count: 1188
tfidf_time: 0.35564422607421875
cosine_time: 0.06715083122253418
find_time: 0.04637408256530762
total_time: 0.46916913986206055

--Run info--
sample_name: longest_docs
threshold: 0.5
pairs_count: 166
tfidf_time: 0.35564422607421875
cosine_time: 0.10275006294250488
find_time: 0.07741093635559082
total_time: 0.5358052253723145

--Run info--
sample_name: longest_docs
threshold: 0.7
pairs_count: 121
tfidf_time: 0.35564422607421875
cosine_time: 0.10658884048461914
find_time: 0.051093101501464844
total_time: 0.5133261680603027

--Run info--
sample_name: longest_docs
threshold: 0.9
pairs_count: 97
tfidf_time: 0.35564422607421875
cosine_time: 0.06276893615722656
find_time: 0.06679916381835938
total_time: 0.4852123260498047



In [22]:
ds_seq.print_first_pair(all_seq_results, "longest_docs", tokenized_samples)

1vimqhdp and pd1g119c have 0.5276 similarity

1vimqhdp:
36th international symposium intensive care emergency medicine brussels belgium 15-18 march 2016 p001 sepsis impairs capillary response within hypoxic capillary decrease erythrocyte oxygen-dependent atp efflux r. m. bateman m. d. sharpe j. e. jagger c. g. elli p002 lower serum immunoglobulin g2 level predispose severe flu j. solé-violán m. lópez-rodríguez e. herrera-ramos j. ruíz-hernández l. borderías j. horcajada n. gonzález-quevedo o. raja m. briones f. rodríguez de castro c. rodríguez gallego p003 brain protective effect intravenous immunoglobulin inhibition complement activation apoptosis rat model sepsis f. esen g. orhun p. ergin ozcan e. senturk c. ugur yilmaz n. orhan n. arican m. kaya m. kucukerden m. giris u. akcan s. bilgic gazioglu e. tuzun p004 adenosine a1 receptor dysfunction associated leukopenia possible mechanism sepsis-induced leukopenia r. riff o. naamani a. douvdevani p005 analysis neutrophil hyper spectral im

### shortest 1000

In [23]:
ds_seq.print_results(all_seq_results, "shortest_docs")

--Run info--
sample_name: shortest_docs
threshold: 0.3
pairs_count: 129
tfidf_time: 0.026515960693359375
cosine_time: 0.007530927658081055
find_time: 0.11735916137695312
total_time: 0.15140604972839355

--Run info--
sample_name: shortest_docs
threshold: 0.5
pairs_count: 62
tfidf_time: 0.026515960693359375
cosine_time: 0.01393580436706543
find_time: 0.042726755142211914
total_time: 0.08317852020263672

--Run info--
sample_name: shortest_docs
threshold: 0.7
pairs_count: 57
tfidf_time: 0.026515960693359375
cosine_time: 0.0063169002532958984
find_time: 0.03412294387817383
total_time: 0.0669558048248291

--Run info--
sample_name: shortest_docs
threshold: 0.9
pairs_count: 54
tfidf_time: 0.026515960693359375
cosine_time: 0.004588127136230469
find_time: 0.03742384910583496
total_time: 0.0685279369354248



In [24]:
ds_seq.print_first_pair(all_seq_results, "shortest_docs", tokenized_samples)

z3c4jvgw and c315r6gu have 1.0000 similarity

z3c4jvgw:
pre-registration undergraduate nurse covid-19 pandemic student worker covid-19 pandemic changed perspective myriad thing sweep across globe one issue forced pay attention risk covid related care entail fact nurse tragically lost life pandemic cook et al 2020 loss life included student nurse ford 2020

c315r6gu:
candida albicans adapting succeed issue cell host microbe lu et al 2013 report redundancy signaling pathway controlling candida albicans filamentation pathogenicity process provide important insight normal commensal human adapts different host microenvironments become highly successful opportunistic pathogen

z3c4jvgw and c315r6gu have 1.0000 similarity

z3c4jvgw:
pre-registration undergraduate nurse covid-19 pandemic student worker covid-19 pandemic changed perspective myriad thing sweep across globe one issue forced pay attention risk covid related care entail fact nurse tragically lost life pandemic cook et al 2020 loss 

### random 1000

In [25]:
ds_seq.print_results(all_seq_results, "random_docs")

--Run info--
sample_name: random_docs
threshold: 0.3
pairs_count: 61
tfidf_time: 0.06726789474487305
cosine_time: 0.013984918594360352
find_time: 0.02706313133239746
total_time: 0.10831594467163086

--Run info--
sample_name: random_docs
threshold: 0.5
pairs_count: 3
tfidf_time: 0.06726789474487305
cosine_time: 0.010812997817993164
find_time: 0.026707887649536133
total_time: 0.10478878021240234

--Run info--
sample_name: random_docs
threshold: 0.7
pairs_count: 0
tfidf_time: 0.06726789474487305
cosine_time: 0.009814977645874023
find_time: 0.024693965911865234
total_time: 0.1017768383026123

--Run info--
sample_name: random_docs
threshold: 0.9
pairs_count: 0
tfidf_time: 0.06726789474487305
cosine_time: 0.010910987854003906
find_time: 0.02400517463684082
total_time: 0.10218405723571777



In [26]:
ds_seq.print_first_pair(all_seq_results, "random_docs", tokenized_samples)

sg0nymkt and hbt4tnpc have 0.3008 similarity

sg0nymkt:
covid-19 united patient provider institutional betrayal health care battle heard believed protected commentary researcher health care consumer medical provider reflect institutional betrayal covid-19 pandemic american canadian health care system example institutional betrayal experienced patient family member well medical provider described although example may evident general public current pandemic represent new problem psycinfo database record c 2020 apa right reserved

hbt4tnpc:
treatment convalescent plasma covid‐19 patient wuhan china discovery severe acute respiratory syndrome coronavirus 2 sars‐cov‐2 outbreak coronavirus disease 2019 covid‐19 causing public health emergency handful literature summarized clinical radiologic feature whereas therapy covid‐19 rather limited order evaluate efficacy convalescent plasma therapy covid‐19 patient timely descriptive study 6 laboratory confirmed covid‐19 patient enrolled received tra

## Sequential APDS with Heuristic

In [27]:
seq_results_dir = os.path.join(samples_dir, "seq_result")
if not os.path.exists(seq_results_dir):
    os.mkdir(seq_results_dir)

seq_results_path_h = os.path.join(seq_results_dir, "seq_results_h.pkl")

In [28]:
#sample: pairs indices, pairs docs id, additional information
all_seq_results_h = ds_seq.exec_seq_apds(tfidf_docs=tfidf_docs,
                                       thresholds=thresholds,
                                       seq_results_path=seq_results_path_h,
                                       samples_dir=samples_dir,
                                       heuristic=True)

100%|██████████| 3/3 [00:02<00:00,  1.15it/s]


### longest 1000

In [29]:
ds_seq.print_results(all_seq_results_h, "longest_docs")

--Run info--
sample_name: longest_docs
threshold: 0.3
pairs_count: 1188
tfidf_time: 0.35564422607421875
cosine_time: 0.06227421760559082
find_time: 0.18860530853271484
total_time: 0.6065237522125244

--Run info--
sample_name: longest_docs
threshold: 0.5
pairs_count: 166
tfidf_time: 0.35564422607421875
cosine_time: 0.11243081092834473
find_time: 0.18112897872924805
total_time: 0.6492040157318115

--Run info--
sample_name: longest_docs
threshold: 0.7
pairs_count: 121
tfidf_time: 0.35564422607421875
cosine_time: 0.0673820972442627
find_time: 0.3209550380706787
total_time: 0.7439813613891602

--Run info--
sample_name: longest_docs
threshold: 0.9
pairs_count: 97
tfidf_time: 0.35564422607421875
cosine_time: 0.0757749080657959
find_time: 0.24147295951843262
total_time: 0.6728920936584473



In [30]:
ds_seq.print_first_pair(all_seq_results_h, "longest_docs", tokenized_samples)

1vimqhdp and pd1g119c have 0.5276 similarity

1vimqhdp:
36th international symposium intensive care emergency medicine brussels belgium 15-18 march 2016 p001 sepsis impairs capillary response within hypoxic capillary decrease erythrocyte oxygen-dependent atp efflux r. m. bateman m. d. sharpe j. e. jagger c. g. elli p002 lower serum immunoglobulin g2 level predispose severe flu j. solé-violán m. lópez-rodríguez e. herrera-ramos j. ruíz-hernández l. borderías j. horcajada n. gonzález-quevedo o. raja m. briones f. rodríguez de castro c. rodríguez gallego p003 brain protective effect intravenous immunoglobulin inhibition complement activation apoptosis rat model sepsis f. esen g. orhun p. ergin ozcan e. senturk c. ugur yilmaz n. orhan n. arican m. kaya m. kucukerden m. giris u. akcan s. bilgic gazioglu e. tuzun p004 adenosine a1 receptor dysfunction associated leukopenia possible mechanism sepsis-induced leukopenia r. riff o. naamani a. douvdevani p005 analysis neutrophil hyper spectral im

### shortest 1000

In [31]:
ds_seq.print_results(all_seq_results_h, "shortest_docs")

--Run info--
sample_name: shortest_docs
threshold: 0.3
pairs_count: 129
tfidf_time: 0.026515960693359375
cosine_time: 0.011932849884033203
find_time: 0.10989499092102051
total_time: 0.14834380149841309

--Run info--
sample_name: shortest_docs
threshold: 0.5
pairs_count: 62
tfidf_time: 0.026515960693359375
cosine_time: 0.00811314582824707
find_time: 0.10724997520446777
total_time: 0.14187908172607422

--Run info--
sample_name: shortest_docs
threshold: 0.7
pairs_count: 57
tfidf_time: 0.026515960693359375
cosine_time: 0.007014036178588867
find_time: 0.14600086212158203
total_time: 0.17953085899353027

--Run info--
sample_name: shortest_docs
threshold: 0.9
pairs_count: 54
tfidf_time: 0.026515960693359375
cosine_time: 0.006190061569213867
find_time: 0.25699496269226074
total_time: 0.289700984954834



In [32]:
ds_seq.print_first_pair(all_seq_results_h, "shortest_docs", tokenized_samples)

z3c4jvgw and c315r6gu have 1.0000 similarity

z3c4jvgw:
pre-registration undergraduate nurse covid-19 pandemic student worker covid-19 pandemic changed perspective myriad thing sweep across globe one issue forced pay attention risk covid related care entail fact nurse tragically lost life pandemic cook et al 2020 loss life included student nurse ford 2020

c315r6gu:
candida albicans adapting succeed issue cell host microbe lu et al 2013 report redundancy signaling pathway controlling candida albicans filamentation pathogenicity process provide important insight normal commensal human adapts different host microenvironments become highly successful opportunistic pathogen

z3c4jvgw and c315r6gu have 1.0000 similarity

z3c4jvgw:
pre-registration undergraduate nurse covid-19 pandemic student worker covid-19 pandemic changed perspective myriad thing sweep across globe one issue forced pay attention risk covid related care entail fact nurse tragically lost life pandemic cook et al 2020 loss 

### random 1000

In [33]:
ds_seq.print_results(all_seq_results_h, "random_docs")

--Run info--
sample_name: random_docs
threshold: 0.3
pairs_count: 61
tfidf_time: 0.06726789474487305
cosine_time: 0.01906871795654297
find_time: 0.14354205131530762
total_time: 0.22987866401672363

--Run info--
sample_name: random_docs
threshold: 0.5
pairs_count: 3
tfidf_time: 0.06726789474487305
cosine_time: 0.010168075561523438
find_time: 0.10049080848693848
total_time: 0.17792677879333496

--Run info--
sample_name: random_docs
threshold: 0.7
pairs_count: 0
tfidf_time: 0.06726789474487305
cosine_time: 0.012320995330810547
find_time: 0.18464088439941406
total_time: 0.26422977447509766

--Run info--
sample_name: random_docs
threshold: 0.9
pairs_count: 0
tfidf_time: 0.06726789474487305
cosine_time: 0.024975299835205078
find_time: 0.15840792655944824
total_time: 0.25065112113952637



In [34]:
ds_seq.print_first_pair(all_seq_results_h, "random_docs", tokenized_samples)

sg0nymkt and hbt4tnpc have 0.3008 similarity

sg0nymkt:
covid-19 united patient provider institutional betrayal health care battle heard believed protected commentary researcher health care consumer medical provider reflect institutional betrayal covid-19 pandemic american canadian health care system example institutional betrayal experienced patient family member well medical provider described although example may evident general public current pandemic represent new problem psycinfo database record c 2020 apa right reserved

hbt4tnpc:
treatment convalescent plasma covid‐19 patient wuhan china discovery severe acute respiratory syndrome coronavirus 2 sars‐cov‐2 outbreak coronavirus disease 2019 covid‐19 causing public health emergency handful literature summarized clinical radiologic feature whereas therapy covid‐19 rather limited order evaluate efficacy convalescent plasma therapy covid‐19 patient timely descriptive study 6 laboratory confirmed covid‐19 patient enrolled received tra

## Setup

In [36]:
thresholds = [0.3, 0.5, 0.7, 0.9]
# n_slices = [4, 7, 10]
n_executors = [4, 7, 9, 12]
n_slices = [8]

## Parallel APDS

In [37]:
mr_results_dir = os.path.join(samples_dir, "mr_result")
if not os.path.exists(mr_results_dir):
    os.mkdir(mr_results_dir)

mr_results_path = os.path.join(mr_results_dir, "mr_results.pkl")

In [38]:
# for collection_name, collection_docs, _ in samples:
all_mr_results = ds_mr.exec_mr_apds(tfidf_docs=tfidf_docs,
                                    thresholds=thresholds,
                                    n_executors=n_executors,
                                    n_slices=n_slices,
                                    mr_results_path=mr_results_path,
                                    samples_dir=samples_dir,
                                    heuristic=False)

  0%|          | 0/3 [00:00<?, ?it/s]


----working on longest_docs, threshold: 0.3----

----workers: 4, slices: 8----


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/01 02:27:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2023-06-01 02:27:58.738 | DEBUG    | src.apds.docsim_mapreduce_spark:create_spark_session:262 - Adding /Users/a/GitHub/All-Pairs-Similarity/src.zip to SparkContext


----Done----



----working on longest_docs, threshold: 0.3----

----workers: 7, slices: 8----


2023-06-01 02:32:47.752 | DEBUG    | src.apds.docsim_mapreduce_spark:create_spark_session:262 - Adding /Users/a/GitHub/All-Pairs-Similarity/src.zip to SparkContext
2023-06-01 02:37:23.103 | DEBUG    | src.apds.docsim_mapreduce_spark:create_spark_session:262 - Adding /Users/a/GitHub/All-Pairs-Similarity/src.zip to SparkContext


...

----Done----



----working on random_docs, threshold: 0.9----

----workers: 9, slices: 8----


2023-06-01 03:32:04.847 | DEBUG    | src.apds.docsim_mapreduce_spark:create_spark_session:262 - Adding /Users/a/GitHub/All-Pairs-Similarity/src.zip to SparkContext


----Done----



----working on random_docs, threshold: 0.9----

----workers: 12, slices: 8----


100%|██████████| 3/3 [1:04:56<00:00, 1298.87s/it]                               

----Done----




In [39]:
all_mr_results

{'longest_docs': {0.3: [([(58, 173, 0.37084156351881337),
     (90, 205, 0.330274701907199),
     (127, 382, 0.38671766951394726),
     (208, 243, 0.34322227537382327),
     (155, 194, 0.3104031712631925),
     (147, 202, 0.37937858548196507),
     (262, 305, 0.38549328668792326),
     (83, 330, 0.31965171141306714),
     (75, 306, 0.30571068536704166),
     (19, 426, 0.3693933142072213),
     (58, 109, 0.35519102332874924),
     (133, 220, 0.32110677893957246),
     (117, 236, 0.352835444401899),
     (393, 440, 0.4277196097326752),
     (13, 52, 0.3325759732307712),
     (162, 325, 0.3557239894869385),
     (191, 382, 0.31327946238237314),
     (312, 363, 0.31946771056160955),
     (237, 244, 0.9983778354618552),
     (61, 324, 0.4188910419699559),
     (240, 243, 0.3047419747320593),
     (109, 116, 0.32825039373897835),
     (74, 93, 0.9813864126076746),
     (220, 231, 0.31015763685885683),
     (62, 153, 0.3127583750032193),
     (90, 109, 0.43317953943402165),
     (41, 88, 0.66

devo estrarre i dati in qualche modo

## Parallel APDS with Heuristic

In [40]:
mr_results_dir = os.path.join(samples_dir, "mr_result")
if not os.path.exists(mr_results_dir):
    os.mkdir(mr_results_dir)

mr_results_path_h = os.path.join(mr_results_dir, "mr_results_h.pkl")

In [ ]:
# for collection_name, collection_docs, _ in samples:
all_mr_results_h = ds_mr.exec_mr_apds(tfidf_docs=tfidf_docs,
                                    thresholds=thresholds,
                                    n_executors=n_executors,
                                    n_slices=n_slices,
                                    mr_results_path=mr_results_path_h,
                                    samples_dir=samples_dir,
                                    heuristic=True)

  0%|          | 0/3 [00:00<?, ?it/s]2023-06-01 03:32:49.748 | DEBUG    | src.apds.docsim_mapreduce_spark:create_spark_session:262 - Adding /Users/a/GitHub/All-Pairs-Similarity/src.zip to SparkContext



----working on longest_docs, threshold: 0.3----

----workers: 4, slices: 8----


2023-06-01 03:37:33.540 | DEBUG    | src.apds.docsim_mapreduce_spark:create_spark_session:262 - Adding /Users/a/GitHub/All-Pairs-Similarity/src.zip to SparkContext


----Done----



----working on longest_docs, threshold: 0.3----

----workers: 7, slices: 8----


2023-06-01 03:42:07.940 | DEBUG    | src.apds.docsim_mapreduce_spark:create_spark_session:262 - Adding /Users/a/GitHub/All-Pairs-Similarity/src.zip to SparkContext


----Done----



----working on longest_docs, threshold: 0.3----

----workers: 9, slices: 8----


In [ ]:
all_mr_results_h